In [ ]:
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from spellchecker import SpellChecker
import os

def load_labeled_data(filename):
    """Load labeled data from a JSON file."""
    try:
        with open(filename, "r", encoding="utf-8") as file:
            data = json.load(file)
        labeled_data = [(item["data"], item["label"]) for item in data]
        return pd.DataFrame(labeled_data, columns=["data", "labels"])
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return None
    except json.JSONDecodeError:
        print(f"Error: Unable to decode JSON file '{filename}'.")
        return None

def train_model(X_train, y_train, vectorizer=None):
    """Train or update a Naive Bayes model."""
    vectorizer = CountVectorizer(analyzer="char", ngram_range=(1, 3))
    X_train_vectorized = vectorizer.fit_transform(X_train)

    model = MultinomialNB()
    model.fit(X_train_vectorized, y_train)

    return model, vectorizer

def save_custom_dictionary(filename, vocabulary):
    """Save custom dictionary as JSON."""
    with open(filename, "w") as file:
        json.dump(list(set(vocabulary)), file)

def main():
    labeled_data_filename = "data_output.json"
    custom_dictionary_filename = "custom_dictionary.json"
    misclassified_samples_filename = "misclassified_samples.json"

    # Load labeled data
    labeled_df = load_labeled_data(labeled_data_filename)
    if labeled_df is None:
        return

    # Initialize an empty dataframe to keep track of all training data
    all_X_train = pd.Series(dtype=str)
    all_y_train = pd.Series(dtype=int)

    while True:  # Continuous learning loop
        # Split the dataset
        X_train, X_test, y_train, y_test = train_test_split(
            labeled_df["data"], labeled_df["labels"], test_size=0.2, random_state=42
        )

        # Combine current training data with all previously seen data
        X_train_combined = pd.concat([all_X_train, X_train], ignore_index=True)
        y_train_combined = pd.concat([all_y_train, y_train], ignore_index=True)

        # Train or update the model
        model, vectorizer = train_model(X_train_combined, y_train_combined)

        # Save custom dictionary as JSON
        save_custom_dictionary(custom_dictionary_filename, vectorizer.get_feature_names())

        # Initialize the spell checker and load the custom dictionary
        spell = SpellChecker()
        spell.word_frequency.load_dictionary(custom_dictionary_filename)

        # Keep track of misclassified samples
        misclassified_samples = []

        words_to_check = ["motday", "Koël"]

        # Use the model to predict correctness
        predictions = model.predict(vectorizer.transform(words_to_check))

        # Save misclassified samples
        for word, prediction in zip(words_to_check, predictions):
            if not prediction:
                misclassified_samples.append(word)

        # Retrain the model using accumulated misclassified samples
        if misclassified_samples:
            X_train_combined = X_train_combined.append(pd.Series(misclassified_samples))
            y_train_combined = y_train_combined.append(pd.Series([0] * len(misclassified_samples)))  # Assuming misclassified samples are incorrect

            model, vectorizer = train_model(X_train_combined, y_train_combined)
            save_custom_dictionary(custom_dictionary_filename, vectorizer.get_feature_names())

        # Clear misclassified samples file
        with open(misclassified_samples_filename, "w") as file:
            json.dump([], file)

        # Alternatively, you can use the correct method directly
        corrected_words = [spell.correction(word) for word in words_to_check]
        print("Corrected Words:", corrected_words)

        # Update all training data with the latest batch
        all_X_train = X_train_combined
        all_y_train = y_train_combined

        # Check for new labeled data and continue the loop if available
        # You may implement logic here to check for new data periodically or interactively
        if not os.path.exists(labeled_data_filename):
            break

if __name__ == "__main__":
    main()


In [ ]:
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from spellchecker import SpellChecker
import os

def load_labeled_data(filename):
    """Load labeled data from a JSON file."""
    try:
        with open(filename, "r", encoding="utf-8") as file:
            data = json.load(file)
        labeled_data = [(item["data"], item["label"]) for item in data]
        return pd.DataFrame(labeled_data, columns=["data", "labels"])
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return None
    except json.JSONDecodeError:
        print(f"Error: Unable to decode JSON file '{filename}'.")
        return None


def train_model(X_train, y_train, vectorizer=None):
    """Train or update a Naive Bayes model."""
    if vectorizer is None:
        vectorizer = CountVectorizer(analyzer="char", ngram_range=(1, 3))
    vectorizer.fit(X_train)  # Fit the vectorizer with training data
    X_train_vectorized = vectorizer.transform(X_train)

    model = MultinomialNB()
    model.fit(X_train_vectorized, y_train)

    return model, vectorizer


def save_custom_dictionary(filename, vocabulary):
    """Save custom dictionary as JSON."""
    with open(filename, "w") as file:
        json.dump(list(set(vocabulary)), file)

def main():
    labeled_data_filename = "data_output.json"
    custom_dictionary_filename = "custom_dictionary.json"
    misclassified_samples_filename = "misclassified_samples.json"

    # Load labeled data
    labeled_df = load_labeled_data(labeled_data_filename)
    if labeled_df is None:
        return

    # Initialize an empty dataframe to keep track of all training data
    all_X_train = pd.Series(dtype=str)
    all_y_train = pd.Series(dtype=int)

    while True:  # Continuous learning loop
        # Split the dataset
        X_train, X_test, y_train, y_test = train_test_split(
            labeled_df["data"], labeled_df["labels"], test_size=0.2, random_state=42
        )

        # Combine current training data with all previously seen data
        X_train_combined = pd.concat([all_X_train, X_train], ignore_index=True)
        y_train_combined = pd.concat([all_y_train, y_train], ignore_index=True)

        # Train or update the model
        model, vectorizer = train_model(X_train_combined, y_train_combined)

        # Save custom dictionary as JSON
        # save_custom_dictionary(custom_dictionary_filename, vectorizer.get_feature_names())
        save_custom_dictionary(custom_dictionary_filename, vectorizer.get_feature_names_out())


        # Initialize the spell checker and load the custom dictionary
        spell = SpellChecker()
        spell.word_frequency.load_dictionary(custom_dictionary_filename)

        # Keep track of misclassified samples
        misclassified_samples = []

        words_to_check = ["motday", "Koël"]

        # Use the model to predict correctness
        predictions = model.predict(vectorizer.transform(words_to_check))

        # Save misclassified samples
        for word, prediction in zip(words_to_check, predictions):
            if not prediction:
                misclassified_samples.append(word)

        # Retrain the model using accumulated misclassified samples
        if misclassified_samples:
            X_train_combined = pd.concat([X_train_combined, pd.Series(misclassified_samples)], ignore_index=True)
            y_train_combined = pd.concat([y_train_combined, pd.Series([0] * len(misclassified_samples))], ignore_index=True)

            # X_train_combined = X_train_combined.concat(pd.Series(misclassified_samples))
            # y_train_combined = y_train_combined.concat(pd.Series([0] * len(misclassified_samples)))  # Assuming misclassified samples are incorrect

            model, vectorizer = train_model(X_train_combined, y_train_combined)
            # save_custom_dictionary(custom_dictionary_filename, vectorizer.get_feature_names())
            # save_custom_dictionary(custom_dictionary_filename, vectorizer.get_feature_names_out())
            save_custom_dictionary(custom_dictionary_filename, vectorizer.get_feature_names_out())


        # Clear misclassified samples file
        with open(misclassified_samples_filename, "w") as file:
            json.dump([], file)

        # Alternatively, you can use the correct method directly
        corrected_words = [spell.correction(word) for word in words_to_check]
        print("Corrected Words:", corrected_words)

        # Update all training data with the latest batch
        all_X_train = X_train_combined
        all_y_train = y_train_combined

        # Check for new labeled data and continue the loop if available
        # You may implement logic here to check for new data periodically or interactively
        if not os.path.exists(labeled_data_filename):
            break

if __name__ == "__main__":
    main()





In [ ]:
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from spellchecker import SpellChecker
import os

def load_labeled_data(filename):
    """Load labeled data from a JSON file."""
    try:
        with open(filename, "r", encoding="utf-8") as file:
            data = json.load(file)
        labeled_data = [(item["data"], item["label"]) for item in data]
        return pd.DataFrame(labeled_data, columns=["data", "labels"])
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return None
    except json.JSONDecodeError:
        print(f"Error: Unable to decode JSON file '{filename}'.")
        return None


def train_model(X_train, y_train, vectorizer=None):
    """Train or update a Naive Bayes model."""
    if vectorizer is None:
        vectorizer = CountVectorizer(analyzer="char", ngram_range=(1, 3))
    vectorizer.fit(X_train)  # Fit the vectorizer with training data
    X_train_vectorized = vectorizer.transform(X_train)

    model = MultinomialNB()
    model.fit(X_train_vectorized, y_train)

    return model, vectorizer


def save_custom_dictionary(filename, vocabulary):
    """Save custom dictionary as JSON."""
    with open(filename, "w") as file:
        json.dump(list(set(vocabulary)), file)

def main():
    labeled_data_filename = "data_output.json"
    custom_dictionary_filename = "custom_dictionary.json"
    misclassified_samples_filename = "misclassified_samples.json"

    # Load labeled data
    labeled_df = load_labeled_data(labeled_data_filename)
    if labeled_df is None:
        return

    # Initialize an empty dataframe to keep track of all training data
    all_X_train = pd.Series(dtype=str)
    all_y_train = pd.Series(dtype=int)

    # Define maximum iterations and misclassification threshold
    max_iterations = 10
    misclassification_threshold = 5
    iteration = 0

    while iteration < max_iterations:
        # Split the dataset
        X_train, X_test, y_train, y_test = train_test_split(
            labeled_df["data"], labeled_df["labels"], test_size=0.2, random_state=42
        )

        # Combine current training data with all previously seen data
        X_train_combined = pd.concat([all_X_train, X_train], ignore_index=True)
        y_train_combined = pd.concat([all_y_train, y_train], ignore_index=True)

        # Train or update the model
        model, vectorizer = train_model(X_train_combined, y_train_combined)

        # Save custom dictionary as JSON
        save_custom_dictionary(custom_dictionary_filename, vectorizer.get_feature_names_out())

        # Initialize the spell checker and load the custom dictionary
        spell = SpellChecker()
        spell.word_frequency.load_dictionary(custom_dictionary_filename)

        # Keep track of misclassified samples
        misclassified_samples = []

        words_to_check = ["motday", "jhelo"]

        # Use the model to predict correctness
        predictions = model.predict(vectorizer.transform(words_to_check))

        # Save misclassified samples
        for word, prediction in zip(words_to_check, predictions):
            if not prediction:
                misclassified_samples.append(word)

        # Retrain the model using accumulated misclassified samples if threshold met
        if misclassified_samples:
            X_train_combined = pd.concat([X_train_combined, pd.Series(misclassified_samples)], ignore_index=True)
            y_train_combined = pd.concat([y_train_combined, pd.Series([0] * len(misclassified_samples))], ignore_index=True)

            model, vectorizer = train_model(X_train_combined, y_train_combined)
            save_custom_dictionary(custom_dictionary_filename, vectorizer.get_feature_names_out())

            iteration += 1

            # Check if misclassification threshold is met
            if len(misclassified_samples) < misclassification_threshold:
                break

        # Clear misclassified samples file
        with open(misclassified_samples_filename, "w") as file:
            json.dump([], file)

        # Alternatively, you can use the correct method directly
        corrected_words = [spell.correction(word) for word in words_to_check]
        print("Corrected Words:", corrected_words)

        # Update all training data with the latest batch
        all_X_train = X_train_combined
        all_y_train = y_train_combined

        # Check for new labeled data and continue the loop if available
        # You may implement logic here to check for new data periodically or interactively
        if not os.path.exists(labeled_data_filename):
            break

    print("Training loop finished.")


if __name__ == "__main__":
    main()


In [22]:
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from spellchecker import SpellChecker
import os

def load_labeled_data(filename):
    """Load labeled data from a JSON file."""
    try:
        with open(filename, "r", encoding="utf-8") as file:
            data = json.load(file)
        labeled_data = [(item["data"], item["label"]) for item in data]
        return pd.DataFrame(labeled_data, columns=["data", "labels"])
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return None
    except json.JSONDecodeError:
        print(f"Error: Unable to decode JSON file '{filename}'.")
        return None


def train_model(X_train, y_train, vectorizer=None):
    """Train or update a Naive Bayes model."""
    if vectorizer is None:
        vectorizer = CountVectorizer(analyzer="char", ngram_range=(1, 3))
    vectorizer.fit(X_train)  # Fit the vectorizer with training data
    X_train_vectorized = vectorizer.transform(X_train)

    model = MultinomialNB()
    model.fit(X_train_vectorized, y_train)

    return model, vectorizer


def save_custom_dictionary(filename, vocabulary):
    """Save custom dictionary as JSON."""
    with open(filename, "w") as file:
        json.dump(list(set(vocabulary)), file)

def main():
    labeled_data_filename = "data_output.json"
    custom_dictionary_filename = "custom_dictionary.json"
    misclassified_samples_filename = "misclassified_samples.json"

    # Load labeled data
    labeled_df = load_labeled_data(labeled_data_filename)
    if labeled_df is None:
        return

    # Initialize an empty dataframe to keep track of all training data
    all_X_train = pd.Series(dtype=str)
    all_y_train = pd.Series(dtype=int)

    # Define maximum iterations and misclassification threshold
    max_iterations = 10
    misclassification_threshold = 5
    iteration = 0

    while iteration < max_iterations:
        # Split the dataset
        X_train, X_test, y_train, y_test = train_test_split(
            labeled_df["data"], labeled_df["labels"], test_size=0.2, random_state=42
        )

        # Combine current training data with all previously seen data
        X_train_combined = pd.concat([all_X_train, X_train], ignore_index=True)
        y_train_combined = pd.concat([all_y_train, y_train], ignore_index=True)

        # Train or update the model
        model, vectorizer = train_model(X_train_combined, y_train_combined)

        # Save custom dictionary as JSON
        save_custom_dictionary(custom_dictionary_filename, vectorizer.get_feature_names_out())

        # Initialize the spell checker and load the custom dictionary
        spell = SpellChecker(language='fr')
        spell.word_frequency.load_dictionary(custom_dictionary_filename)

        # Keep track of misclassified samples
        misclassified_samples = []

        words_to_check = ["motday", "Koël"]

        # Use the model to predict correctness
        predictions = model.predict(vectorizer.transform(words_to_check))

        # Save misclassified samples
        for word, prediction in zip(words_to_check, predictions):
            if not prediction:
                misclassified_samples.append(word)

        # Retrain the model using accumulated misclassified samples if threshold met
        if misclassified_samples:
            X_train_combined = pd.concat([X_train_combined, pd.Series(misclassified_samples)], ignore_index=True)
            y_train_combined = pd.concat([y_train_combined, pd.Series([0] * len(misclassified_samples))], ignore_index=True)

            model, vectorizer = train_model(X_train_combined, y_train_combined)
            save_custom_dictionary(custom_dictionary_filename, vectorizer.get_feature_names_out())

            iteration += 1

            # Check if misclassification threshold is met
            if len(misclassified_samples) < misclassification_threshold:
                break

        # Clear misclassified samples file
        with open(misclassified_samples_filename, "w") as file:
            json.dump([], file)

        # Alternatively, you can use the correct method directly
        corrected_words = [spell.correction(word) for word in words_to_check]
        print("Corrected Words:", corrected_words)

        # Update all training data with the latest batch
        all_X_train = X_train_combined
        all_y_train = y_train_combined

        # Check for new labeled data and continue the loop if available
        # You may implement logic here to check for new data periodically or interactively
        if not os.path.exists(labeled_data_filename):
            break

    print("Training loop finished.")


if __name__ == "__main__":
    main()


Training loop finished.


In [ ]:
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from spellchecker import SpellChecker
import os

def load_labeled_data(filename):
    """Load labeled data from a JSON file."""
    try:
        with open(filename, "r", encoding="utf-8") as file:
            data = json.load(file)
        labeled_data = [(item["data"], item["label"]) for item in data]
        return pd.DataFrame(labeled_data, columns=["data", "labels"])
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return None
    except json.JSONDecodeError:
        print(f"Error: Unable to decode JSON file '{filename}'.")
        return None


def train_model(X_train, y_train, vectorizer=None):
    """Train or update a Naive Bayes model."""
    if vectorizer is None:
        vectorizer = CountVectorizer(analyzer="char", ngram_range=(1, 3))
    vectorizer.fit(X_train)  # Fit the vectorizer with training data
    X_train_vectorized = vectorizer.transform(X_train)

    model = MultinomialNB()
    model.fit(X_train_vectorized, y_train)

    return model, vectorizer


def save_custom_dictionary(filename, vocabulary):
    """Save custom dictionary as JSON."""
    with open(filename, "w") as file:
        json.dump(list(set(vocabulary)), file)

def main():
    labeled_data_filename = "data_output.json"
    custom_dictionary_filename = "custom_dictionary.json"
    misclassified_samples_filename = "misclassified_samples.json"

    # Load labeled data
    labeled_df = load_labeled_data(labeled_data_filename)
    if labeled_df is None:
        return

    # Initialize an empty dataframe to keep track of all training data
    all_X_train = pd.Series(dtype=str)
    all_y_train = pd.Series(dtype=int)

    # Define maximum iterations and misclassification threshold
    max_iterations = 10
    misclassification_threshold = 5
    iteration = 0

    while iteration < max_iterations:
        # Split the dataset
        X_train, X_test, y_train, y_test = train_test_split(
            labeled_df["data"], labeled_df["labels"], test_size=0.2, random_state=42
        )

        # Combine current training data with all previously seen data
        X_train_combined = pd.concat([all_X_train, X_train], ignore_index=True)
        y_train_combined = pd.concat([all_y_train, y_train], ignore_index=True)

        # Train or update the model
        model, vectorizer = train_model(X_train_combined, y_train_combined)

        # Save custom dictionary as JSON
        save_custom_dictionary(custom_dictionary_filename, vectorizer.get_feature_names_out())

        # Initialize the spell checker and load the custom dictionary
        spell = SpellChecker()
        spell.word_frequency.load_dictionary(custom_dictionary_filename)

        # Keep track of misclassified samples
        misclassified_samples = []

        words_to_check = ["motday", "Koël"]

        # Use the model to predict correctness
        predictions = model.predict(vectorizer.transform(words_to_check))

        # Save misclassified samples
        for word, prediction in zip(words_to_check, predictions):
            if not prediction:
                misclassified_samples.append(word)

        # Retrain the model using accumulated misclassified samples if threshold met
        if misclassified_samples:
            X_train_combined = pd.concat([X_train_combined, pd.Series(misclassified_samples)], ignore_index=True)
            y_train_combined = pd.concat([y_train_combined, pd.Series([0] * len(misclassified_samples))], ignore_index=True)

            model, vectorizer = train_model(X_train_combined, y_train_combined)
            save_custom_dictionary(custom_dictionary_filename, vectorizer.get_feature_names_out())

            iteration += 1

            # Check if misclassification threshold is met
            if len(misclassified_samples) < misclassification_threshold:
                break
        # Load misclassified samples
        with open(misclassified_samples_filename, "r") as file:
            misclassified_samples = json.load(file)

        # Clear misclassified samples file
        with open(misclassified_samples_filename, "w") as file:
            json.dump([], file)

        # Alternatively, you can use the correct method directly
        corrected_words = [spell.correction(word) for word in words_to_check]
        print("Corrected Words:", corrected_words)

        # Update all training data with the latest batch
        all_X_train = X_train_combined
        all_y_train = y_train_combined

        # Check for new labeled data and continue the loop if available
        # You may implement logic here to check for new data periodically or interactively
        if not os.path.exists(labeled_data_filename):
            break

    print("Training loop finished.")


if __name__ == "__main__":
    main()


In [ ]:
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from spellchecker import SpellChecker
import os

def load_labeled_data(filename):
    """Load labeled data from a JSON file."""
    try:
        with open(filename, "r", encoding="utf-8") as file:
            data = json.load(file)
        labeled_data = [(item["data"], item["label"]) for item in data]
        return pd.DataFrame(labeled_data, columns=["data", "labels"])
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return None
    except json.JSONDecodeError:
        print(f"Error: Unable to decode JSON file '{filename}'.")
        return None


def train_model(X_train, y_train, vectorizer=None):
    """Train or update a Naive Bayes model."""
    if vectorizer is None:
        vectorizer = CountVectorizer(analyzer="char", ngram_range=(1, 3))
    vectorizer.fit(X_train)  # Fit the vectorizer with training data
    X_train_vectorized = vectorizer.transform(X_train)

    model = MultinomialNB()
    model.fit(X_train_vectorized, y_train)

    return model, vectorizer


def save_custom_dictionary(filename, vocabulary):
    """Save custom dictionary as JSON."""
    with open(filename, "w") as file:
        json.dump(list(set(vocabulary)), file)

def main():
    labeled_data_filename = "data_output.json"
    custom_dictionary_filename = "custom_dictionary.json"
    misclassified_samples_filename = "misclassified_samples.json"

    # Load labeled data
    labeled_df = load_labeled_data(labeled_data_filename)
    if labeled_df is None:
        return

    # Initialize an empty dataframe to keep track of all training data
    all_X_train = pd.Series(dtype=str)
    all_y_train = pd.Series(dtype=int)

    # Define maximum iterations and misclassification threshold
    max_iterations = 10
    misclassification_threshold = 5
    iteration = 0

    while iteration < max_iterations:
        # Split the dataset
        X_train, X_test, y_train, y_test = train_test_split(
            labeled_df["data"], labeled_df["labels"], test_size=0.2, random_state=42
        )

        # Combine current training data with all previously seen data
        X_train_combined = pd.concat([all_X_train, X_train], ignore_index=True)
        y_train_combined = pd.concat([all_y_train, y_train], ignore_index=True)

        # Train or update the model
        model, vectorizer = train_model(X_train_combined, y_train_combined)

        # Save custom dictionary as JSON
        save_custom_dictionary(custom_dictionary_filename, vectorizer.get_feature_names_out())

        # Initialize the spell checker and load the custom dictionary
        spell = SpellChecker()
        spell.word_frequency.load_dictionary(custom_dictionary_filename)

        # Print corrected words based on spell checker
        words_to_check = ["motday", "Koël"]
        corrected_words = [spell.correction(word) for word in words_to_check]
        print("Corrected Words:", corrected_words)

        # Keep track of misclassified samples
        misclassified_samples = []

        # Use the model to predict correctness
        predictions = model.predict(vectorizer.transform(words_to_check))

        # Save misclassified samples
        for word, prediction in zip(words_to_check, predictions):
            if not prediction:
                misclassified_samples.append(word)

        # Retrain the model using accumulated misclassified samples if threshold met
        if misclassified_samples:
            X_train_combined = pd.concat([X_train_combined, pd.Series(misclassified_samples)], ignore_index=True)
            y_train_combined = pd.concat([y_train_combined, pd.Series([0] * len(misclassified_samples))], ignore_index=True)

            model, vectorizer = train_model(X_train_combined, y_train_combined)
            save_custom_dictionary(custom_dictionary_filename, vectorizer.get_feature_names_out())

            iteration += 1

            # Check if misclassification threshold is met
            if len(misclassified_samples) < misclassification_threshold:
                break
        
        # Load misclassified samples
        with open(misclassified_samples_filename, "r") as file:
            misclassified_samples = json.load(file)

        print("Misclassified Samples:", misclassified_samples)  # Debugging
        
        # Clear misclassified samples file
        with open(misclassified_samples_filename, "w") as file:
            json.dump([], file)

        # Update all training data with the latest batch
        all_X_train = X_train_combined
        all_y_train = y_train_combined

        # Check for new labeled data and continue the loop if available
        # You may implement logic here to check for new data periodically or interactively
        if not os.path.exists(labeled_data_filename):
            break

    print("Training loop finished.")


if __name__ == "__main__":
    main()


In [ ]:
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from spellchecker import SpellChecker
import os

def load_labeled_data(filename):
    """Load labeled data from a JSON file."""
    try:
        with open(filename, "r", encoding="utf-8") as file:
            data = json.load(file)
        labeled_data = [(item["data"], item["label"]) for item in data]
        return pd.DataFrame(labeled_data, columns=["data", "labels"])
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return None
    except json.JSONDecodeError:
        print(f"Error: Unable to decode JSON file '{filename}'.")
        return None


def train_model(X_train, y_train, vectorizer=None):
    """Train or update a Naive Bayes model."""
    if vectorizer is None:
        vectorizer = CountVectorizer(analyzer="char", ngram_range=(1, 3))
    vectorizer.fit(X_train)  # Fit the vectorizer with training data
    X_train_vectorized = vectorizer.transform(X_train)

    model = MultinomialNB()
    model.fit(X_train_vectorized, y_train)

    return model, vectorizer


def save_custom_dictionary(filename, vocabulary):
    """Save custom dictionary as JSON."""
    with open(filename, "w") as file:
        json.dump(list(set(vocabulary)), file)

def main():
    labeled_data_filename = "data_output.json"
    custom_dictionary_filename = "custom_dictionary.json"
    misclassified_samples_filename = "misclassified_samples.json"

    # Load labeled data
    labeled_df = load_labeled_data(labeled_data_filename)
    if labeled_df is None:
        return

    # Initialize an empty dataframe to keep track of all training data
    all_X_train = pd.Series(dtype=str)
    all_y_train = pd.Series(dtype=int)

    # Define maximum iterations and misclassification threshold
    max_iterations = 10
    misclassification_threshold = 5
    iteration = 0

    while iteration < max_iterations:
        # Split the dataset
        X_train, X_test, y_train, y_test = train_test_split(
            labeled_df["data"], labeled_df["labels"], test_size=0.2, random_state=42
        )

        # Combine current training data with all previously seen data
        X_train_combined = pd.concat([all_X_train, X_train], ignore_index=True)
        y_train_combined = pd.concat([all_y_train, y_train], ignore_index=True)

        # Train or update the model
        model, vectorizer = train_model(X_train_combined, y_train_combined)

        # Save custom dictionary as JSON
        save_custom_dictionary(custom_dictionary_filename, vectorizer.get_feature_names_out())

        # Initialize the spell checker and load the custom dictionary
        spell = SpellChecker()
        spell.word_frequency.load_dictionary(custom_dictionary_filename)

        # Print corrected words and suggestions based on spell checker
        words_to_check = ["motday", "hgots"]
        corrected_words = []
        suggestions = []
        for word in words_to_check:
            corrected_word = spell.correction(word)
            suggestions.append(spell.candidates(word))
            corrected_words.append(corrected_word)
        
        print("Corrected Words:", corrected_words)
        print("Suggestions:", suggestions)

        # Keep track of misclassified samples
        misclassified_samples = []

        # Use the model to predict correctness
        predictions = model.predict(vectorizer.transform(words_to_check))

        # Save misclassified samples
        for word, prediction in zip(words_to_check, predictions):
            if not prediction:
                misclassified_samples.append(word)

        # Retrain the model using accumulated misclassified samples if threshold met
        if misclassified_samples:
            X_train_combined = pd.concat([X_train_combined, pd.Series(misclassified_samples)], ignore_index=True)
            y_train_combined = pd.concat([y_train_combined, pd.Series([0] * len(misclassified_samples))], ignore_index=True)

            model, vectorizer = train_model(X_train_combined, y_train_combined)
            save_custom_dictionary(custom_dictionary_filename, vectorizer.get_feature_names_out())

            iteration += 1

            # Check if misclassification threshold is met
            if len(misclassified_samples) < misclassification_threshold:
                break
        
        # Load misclassified samples
        with open(misclassified_samples_filename, "r") as file:
            misclassified_samples = json.load(file)

        print("Misclassified Samples:", misclassified_samples)  # Debugging
        
        # Clear misclassified samples file
        with open(misclassified_samples_filename, "w") as file:
            json.dump([], file)

        # Update all training data with the latest batch
        all_X_train = X_train_combined
        all_y_train = y_train_combined

        # Check for new labeled data and continue the loop if available
        # You may implement logic here to check for new data periodically or interactively
        if not os.path.exists(labeled_data_filename):
            break

    print("Training loop finished.")


if __name__ == "__main__":
    main()


In [8]:
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from spellchecker import SpellChecker
import os


def load_labeled_data(filename):
    """Load labeled data from a JSON file."""
    try:
        with open(filename, "r", encoding="utf-8") as file:
            data = json.load(file)
        labeled_data = [(item["data"], item["label"]) for item in data]
        return pd.DataFrame(labeled_data, columns=["data", "labels"])
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return None
    except json.JSONDecodeError:
        print(f"Error: Unable to decode JSON file '{filename}'.")
        return None


def train_model(X_train, y_train, vectorizer=None):
    """Train or update a Naive Bayes model."""
    if vectorizer is None:
        vectorizer = CountVectorizer(analyzer="char", ngram_range=(1, 3))
    vectorizer.fit(X_train)  # Fit the vectorizer with training data
    X_train_vectorized = vectorizer.transform(X_train)

    model = MultinomialNB()
    model.fit(X_train_vectorized, y_train)

    return model, vectorizer


def save_custom_dictionary(filename, vocabulary):
    """Save custom dictionary as JSON."""
    with open(filename, "w") as file:
        json.dump(list(set(vocabulary)), file)


def main():
    labeled_data_filename = "data_output.json"
    custom_dictionary_filename = "custom_dictionary.json"
    misclassified_samples_filename = "misclassified_samples.json"

    # Initialize the spell checker and load the custom dictionary
    spell = SpellChecker()
    if os.path.exists(custom_dictionary_filename):
        spell.word_frequency.load_dictionary(custom_dictionary_filename)

    # Load labeled data
    labeled_df = load_labeled_data(labeled_data_filename)
    if labeled_df is None:
        return

    # Initialize an empty dataframe to keep track of all training data
    all_X_train = pd.Series(dtype=str)
    all_y_train = pd.Series(dtype=int)

    # Define maximum iterations and misclassification threshold
    max_epochs = 20
    misclassification_threshold = 5
    epoch = 0

    while epoch < max_epochs:
        # Split the dataset
        X_train, X_test, y_train, y_test = train_test_split(
            labeled_df["data"], labeled_df["labels"], test_size=0.2, random_state=42
        )

        # Combine current training data with all previously seen data
        X_train_combined = pd.concat([all_X_train, X_train], ignore_index=True)
        y_train_combined = pd.concat([all_y_train, y_train], ignore_index=True)

        # Train or update the model
        model, vectorizer = train_model(X_train_combined, y_train_combined)

        # Save custom dictionary as JSON
        save_custom_dictionary(
            custom_dictionary_filename, vectorizer.get_feature_names_out()
        )

        # Print corrected words and suggestions based on spell checker
        words_to_check = ["motday", "hgost", "mney"]
        corrected_words = [spell.correction(word) for word in words_to_check]
        suggestions = [spell.candidates(word) for word in words_to_check]
        print("Corrected Words:", corrected_words)
        print("Suggestions:", suggestions)

        # Keep track of misclassified samples
        misclassified_samples = []

        # Use the model to predict correctness
        predictions = model.predict(vectorizer.transform(words_to_check))

        # Save misclassified samples
        for word, prediction in zip(words_to_check, predictions):
            if not prediction:
                misclassified_samples.append(word)

        # Accumulate misclassified samples
        with open(misclassified_samples_filename, "r") as file:
            prev_misclassified_samples = json.load(file)
        misclassified_samples.extend(prev_misclassified_samples)

        # Retrain the model using accumulated misclassified samples
        if misclassified_samples:
            X_train_combined = pd.concat(
                [X_train_combined, pd.Series(misclassified_samples)], ignore_index=True
            )
            y_train_combined = pd.concat(
                [y_train_combined, pd.Series([0] * len(misclassified_samples))],
                ignore_index=True,
            )

            model, vectorizer = train_model(X_train_combined, y_train_combined)
            save_custom_dictionary(
                custom_dictionary_filename, vectorizer.get_feature_names_out()
            )

            # Clear misclassified samples file
            with open(misclassified_samples_filename, "w") as file:
                json.dump([], file)

        epoch += 1

        # Check if misclassification threshold is met
        if len(misclassified_samples) < misclassification_threshold:
            break

        # Update all training data with the latest batch
        all_X_train = X_train_combined
        all_y_train = y_train_combined

        # Check for new labeled data and continue the loop if available
        # You may implement logic here to check for new data periodically or interactively
        if not os.path.exists(labeled_data_filename):
            break

    print("Training loop finished.")


if __name__ == "__main__":
    main()

Corrected Words: ['today', 'ghost', 'money']
Suggestions: [{'moldy', 'modal', 'moray', 'mothy', 'moody', 'midday', 'today', 'mayday', 'motley'}, {'host', 'ghost'}, {'money'}]
Training loop finished.


In [16]:
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from spellchecker import SpellChecker
import os


def load_labeled_data(filename):
    """Load labeled data from a JSON file."""
    try:
        with open(filename, "r", encoding="utf-8") as file:
            data = json.load(file)
        labeled_data = [(item["data"], item["label"]) for item in data]
        return pd.DataFrame(labeled_data, columns=["data", "labels"])
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return None
    except json.JSONDecodeError:
        print(f"Error: Unable to decode JSON file '{filename}'.")
        return None


def train_model(X_train, y_train, vectorizer=None):
    """Train or update a Naive Bayes model."""
    if vectorizer is None:
        vectorizer = CountVectorizer(analyzer="char", ngram_range=(1, 3))
    vectorizer.fit(X_train)  # Fit the vectorizer with training data
    X_train_vectorized = vectorizer.transform(X_train)

    model = MultinomialNB()
    model.fit(X_train_vectorized, y_train)

    return model, vectorizer


def save_custom_dictionary(filename, vocabulary):
    """Save custom dictionary as JSON."""
    with open(filename, "w") as file:
        json.dump(list(set(vocabulary)), file)


def main():
    labeled_data_filename = "data_output.json"
    custom_dictionary_filename = "custom_dictionary.json"
    misclassified_samples_filename = "misclassified_samples.json"

    # Initialize the spell checker and load the custom dictionary
    spell = SpellChecker()
    if os.path.exists(custom_dictionary_filename):
        spell.word_frequency.load_dictionary(custom_dictionary_filename)

    # Load labeled data
    labeled_df = load_labeled_data(labeled_data_filename)
    if labeled_df is None:
        return

    # Initialize an empty dataframe to keep track of all training data
    all_X_train = pd.Series(dtype=str)
    all_y_train = pd.Series(dtype=int)

    # Define maximum iterations and misclassification threshold
    max_epochs = 20
    misclassification_threshold = 5
    epoch = 0

    while epoch < max_epochs:
        # Split the dataset
        X_train, X_test, y_train, y_test = train_test_split(
            labeled_df["data"], labeled_df["labels"], test_size=0.2, random_state=42
        )

        # Combine current training data with all previously seen data
        X_train_combined = pd.concat([all_X_train, X_train], ignore_index=True)
        y_train_combined = pd.concat([all_y_train, y_train], ignore_index=True)

        # Train or update the model
        model, vectorizer = train_model(X_train_combined, y_train_combined)

        # Save custom dictionary as JSON
        save_custom_dictionary(
            custom_dictionary_filename, vectorizer.get_feature_names_out()
        )

        # Print corrected words and suggestions based on spell checker
        words_to_check = ["motday", "hgost", "mney"]
        corrected_words = [spell.correction(word) for word in words_to_check]
        suggestions = [spell.candidates(word) for word in words_to_check]
        print("Corrected Words:", corrected_words)
        print("Suggestions:", suggestions)

        # Keep track of misclassified samples
        misclassified_samples = []

        # Use the model to predict correctness
        predictions = model.predict(vectorizer.transform(words_to_check))

        # Save misclassified samples
        for word, prediction in zip(words_to_check, predictions):
            if not prediction:
                misclassified_samples.append(word)

        # Accumulate misclassified samples
        with open(misclassified_samples_filename, "r") as file:
            prev_misclassified_samples = json.load(file)
        misclassified_samples.extend(prev_misclassified_samples)

        # Retrain the model using accumulated misclassified samples
        if misclassified_samples:
            X_train_combined = pd.concat(
                [X_train_combined, pd.Series(misclassified_samples)], ignore_index=True
            )
            y_train_combined = pd.concat(
                [y_train_combined, pd.Series([0] * len(misclassified_samples))],
                ignore_index=True,
            )

            model, vectorizer = train_model(X_train_combined, y_train_combined)
            save_custom_dictionary(
                custom_dictionary_filename, vectorizer.get_feature_names_out()
            )

            # Clear misclassified samples file
            with open(misclassified_samples_filename, "w") as file:
                json.dump([], file)

        epoch += 1

        # Check if misclassification threshold is met
        if len(misclassified_samples) < misclassification_threshold:
            break

        # Update all training data with the latest batch
        all_X_train = X_train_combined
        all_y_train = y_train_combined

        # Check for new labeled data and continue the loop if available
        # You may implement logic here to check for new data periodically or interactively
        if not os.path.exists(labeled_data_filename):
            break

    print("Training loop finished.")


if __name__ == "__main__":
    main()

Corrected Words: ['today', 'ghost', 'money']
Suggestions: [{'moldy', 'modal', 'moray', 'mothy', 'moody', 'midday', 'today', 'mayday', 'motley'}, {'host', 'ghost'}, {'money'}]
Training loop finished.


In [17]:
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from spellchecker import SpellChecker
import os

def load_labeled_data(filename):
    """Load labeled data from a JSON file."""
    try:
        with open(filename, "r", encoding="utf-8") as file:
            data = json.load(file)
        labeled_data = [(item["data"], item["label"]) for item in data]
        return pd.DataFrame(labeled_data, columns=["data", "labels"])
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return None
    except json.JSONDecodeError:
        print(f"Error: Unable to decode JSON file '{filename}'.")
        return None

def train_model(X_train, y_train, vectorizer=None):
    """Train or update a Naive Bayes model."""
    if vectorizer is None:
        vectorizer = CountVectorizer(analyzer="char", ngram_range=(1, 3))
    vectorizer.fit(X_train)  # Fit the vectorizer with training data
    X_train_vectorized = vectorizer.transform(X_train)

    model = MultinomialNB()
    model.fit(X_train_vectorized, y_train)

    return model, vectorizer

def save_custom_dictionary(filename, vocabulary):
    """Save custom dictionary as JSON."""
    with open(filename, "w") as file:
        json.dump(list(set(vocabulary)), file)

def check_spelling(spell, words_to_check):
    """Check spelling and get suggestions."""
    corrected_words = [spell.correction(word) for word in words_to_check]
    suggestions = [spell.candidates(word) for word in words_to_check]
    return corrected_words, suggestions

def main():
    labeled_data_filename = "data_output.json"
    custom_dictionary_filename = "custom_dictionary.json"
    misclassified_samples_filename = "misclassified_samples.json"

    # Initialize the spell checker and load the custom dictionary
    spell = SpellChecker()
    if os.path.exists(custom_dictionary_filename):
        spell.word_frequency.load_dictionary(custom_dictionary_filename)

    # Load labeled data
    labeled_df = load_labeled_data(labeled_data_filename)
    if labeled_df is None:
        return

    # Initialize an empty dataframe to keep track of all training data
    all_X_train = pd.Series(dtype=str)
    all_y_train = pd.Series(dtype=int)

    # Define maximum iterations and misclassification threshold
    max_epochs = 20
    misclassification_threshold = 5
    epoch = 0
    misclassified_samples = []

    while epoch < max_epochs:
        # Split the dataset
        X_train, X_test, y_train, y_test = train_test_split(
            labeled_df["data"], labeled_df["labels"], test_size=0.2, random_state=42
        )

        # Combine current training data with all previously seen data
        X_train_combined = pd.concat([all_X_train, X_train], ignore_index=True)
        y_train_combined = pd.concat([all_y_train, y_train], ignore_index=True)

        # Train or update the model
        model, vectorizer = train_model(X_train_combined, y_train_combined)

        # Print corrected words and suggestions based on spell checker
        words_to_check = ["motday", "hgost", "mney"]
        corrected_words, suggestions = check_spelling(spell, words_to_check)
        print("Corrected Words:", corrected_words)
        print("Suggestions:", suggestions)

        # Use the model to predict correctness
        predictions = model.predict(vectorizer.transform(words_to_check))

        # Save misclassified samples
        for word, prediction in zip(words_to_check, predictions):
            if not prediction:
                misclassified_samples.append(word)

        epoch += 1

        # Check if misclassification threshold is met
        if len(misclassified_samples) >= misclassification_threshold:
            break

        # Update all training data with the latest batch
        all_X_train = X_train_combined
        all_y_train = y_train_combined

        # Check for new labeled data and continue the loop if available
        # You may implement logic here to check for new data periodically or interactively
        if not os.path.exists(labeled_data_filename):
            break

    # Save custom dictionary as JSON
    save_custom_dictionary(
        custom_dictionary_filename, vectorizer.get_feature_names_out()
    )

    # Save misclassified samples
    with open(misclassified_samples_filename, "w") as file:
        json.dump(misclassified_samples, file)

    print("Training loop finished.")

if __name__ == "__main__":
    main()


Corrected Words: ['today', 'ghost', 'money']
Suggestions: [{'moldy', 'modal', 'moray', 'mothy', 'moody', 'midday', 'today', 'mayday', 'motley'}, {'host', 'ghost'}, {'money'}]
Corrected Words: ['today', 'ghost', 'money']
Suggestions: [{'moldy', 'modal', 'moray', 'mothy', 'moody', 'midday', 'today', 'mayday', 'motley'}, {'host', 'ghost'}, {'money'}]
Corrected Words: ['today', 'ghost', 'money']
Suggestions: [{'moldy', 'modal', 'moray', 'mothy', 'moody', 'midday', 'today', 'mayday', 'motley'}, {'host', 'ghost'}, {'money'}]
Corrected Words: ['today', 'ghost', 'money']
Suggestions: [{'moldy', 'modal', 'moray', 'mothy', 'moody', 'midday', 'today', 'mayday', 'motley'}, {'host', 'ghost'}, {'money'}]
Corrected Words: ['today', 'ghost', 'money']
Suggestions: [{'moldy', 'modal', 'moray', 'mothy', 'moody', 'midday', 'today', 'mayday', 'motley'}, {'host', 'ghost'}, {'money'}]
Corrected Words: ['today', 'ghost', 'money']
Suggestions: [{'moldy', 'modal', 'moray', 'mothy', 'moody', 'midday', 'today', '

In [19]:
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from spellchecker import SpellChecker
import os

def load_labeled_data(filename):
    """Load labeled data from a JSON file."""
    try:
        with open(filename, "r", encoding="utf-8") as file:
            data = json.load(file)
        labeled_data = [(item["data"], item["label"]) for item in data]
        return pd.DataFrame(labeled_data, columns=["data", "labels"])
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return None
    except json.JSONDecodeError:
        print(f"Error: Unable to decode JSON file '{filename}'.")
        return None

def train_model(X_train, y_train, vectorizer=None):
    """Train or update a Naive Bayes model."""
    if vectorizer is None:
        vectorizer = CountVectorizer(analyzer="char", ngram_range=(1, 3))
    vectorizer.fit(X_train)  # Fit the vectorizer with training data
    X_train_vectorized = vectorizer.transform(X_train)

    model = MultinomialNB()
    model.fit(X_train_vectorized, y_train)

    return model, vectorizer

def save_custom_dictionary(filename, vocabulary):
    """Save custom dictionary as JSON."""
    with open(filename, "w") as file:
        json.dump(list(set(vocabulary)), file)

def check_spelling(spell, words_to_check):
    """Check spelling and get suggestions."""
    corrected_words = [spell.correction(word) for word in words_to_check]
    suggestions = [spell.candidates(word) for word in words_to_check]
    return corrected_words, suggestions

def main():
    labeled_data_filename = "data_output.json"
    custom_dictionary_filename = "custom_dictionary.json"
    misclassified_samples_filename = "misclassified_samples.json"

    # Initialize the spell checker and load the custom dictionary
    spell = SpellChecker()
    if os.path.exists(custom_dictionary_filename):
        spell.word_frequency.load_dictionary(custom_dictionary_filename)

    # Initialize an empty list to keep track of misclassified samples
    misclassified_samples = []
    misclassification_threshold = 5
    epochs = 0

    # Initialize empty DataFrames to store all training data
    all_X_train = pd.Series(dtype=str)
    all_y_train = pd.Series(dtype=int)

    while epochs < 20:
        labeled_df = load_labeled_data(labeled_data_filename)
        if labeled_df is None:
            return

        X_train, X_test, y_train, y_test = train_test_split(
            labeled_df["data"], labeled_df["labels"], test_size=0.2, random_state=42
        )

        all_X_train = pd.concat([all_X_train, X_train], ignore_index=True)
        all_y_train = pd.concat([all_y_train, y_train], ignore_index=True)

        model, vectorizer = train_model(all_X_train, all_y_train)

        # Check for misclassified samples
        words_to_check = ["motday", "hgost", "mney"]
        predictions = model.predict(vectorizer.transform(words_to_check))
        misclassified_samples.extend([word for word, prediction in zip(words_to_check, predictions) if not prediction])

        if len(misclassified_samples) >= misclassification_threshold:
            break

        epochs += 1

    # Save custom dictionary incrementally
    new_vocabulary = vectorizer.get_feature_names_out()
    if os.path.exists(custom_dictionary_filename):
        with open(custom_dictionary_filename, "r") as file:
            existing_vocabulary = set(json.load(file))
        new_vocabulary = existing_vocabulary.union(new_vocabulary)
    with open(custom_dictionary_filename, "w") as file:
        json.dump(list(new_vocabulary), file)

    # Save misclassified samples
    with open(misclassified_samples_filename, "w") as file:
        json.dump(misclassified_samples, file)

    print("Training loop finished.")

if __name__ == "__main__":
    main()


Training loop finished.


In [25]:
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from spellchecker import SpellChecker
import os


def load_labeled_data(filename):
    """Load labeled data from a JSON file."""
    try:
        with open(filename, "r", encoding="utf-8") as file:
            data = json.load(file)
            labeled_data = [(item.get("data"), item.get("label")) for item in data]
            return pd.DataFrame(labeled_data, columns=["data", "labels"])
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
    except json.JSONDecodeError as e:
        print(f"Error: Unable to decode JSON file '{filename}': {e}")
    except Exception as e:
        print(f"Error: An unexpected error occurred: {e}")

    return None


def train_model(X_train, y_train, vectorizer=None):
    """Train or update a Naive Bayes model."""
    if vectorizer is None:
        vectorizer = CountVectorizer(analyzer="char", ngram_range=(1, 3))
    vectorizer.fit(X_train)  # Fit the vectorizer with training data
    X_train_vectorized = vectorizer.transform(X_train)

    model = MultinomialNB()
    model.fit(X_train_vectorized, y_train)

    return model, vectorizer


def save_custom_dictionary(filename, vocabulary):
    """Save custom dictionary as JSON."""
    with open(filename, "w") as file:
        json.dump(list(set(vocabulary)), file)


def check_spelling(spell, words_to_check):
    """Check spelling and get suggestions."""
    corrected_words = [spell.correction(word) for word in words_to_check]
    suggestions = [spell.candidates(word) for word in words_to_check]
    return corrected_words, suggestions


def main():
    labeled_data_filename = "data_output.json"
    custom_dictionary_filename = "custom_dictionary.json"
    misclassified_samples_filename = "misclassified_samples.json"

    # Initialize the spell checker and load the custom dictionary
    spell = SpellChecker(language="en")
    if os.path.exists(custom_dictionary_filename):
        spell.word_frequency.load_dictionary(custom_dictionary_filename)

    # Initialize an empty list to keep track of misclassified samples
    misclassified_samples = []
    misclassification_threshold = 5
    epochs = 0

    # Initialize empty DataFrames to store all training data
    all_X_train = pd.Series(dtype=str)
    all_y_train = pd.Series(dtype=int)

    while epochs < 20:
        labeled_df = load_labeled_data(labeled_data_filename)
        if labeled_df is None:
            return

        X_train, X_test, y_train, y_test = train_test_split(
            labeled_df["data"], labeled_df["labels"], test_size=0.2, random_state=42
        )

        all_X_train = pd.concat([all_X_train, X_train], ignore_index=True)
        all_y_train = pd.concat([all_y_train, y_train], ignore_index=True)

        model, vectorizer = train_model(all_X_train, all_y_train)

        # Check for misclassified samples
        words_to_check = ["motday", "hgost", "mney"]
        predictions = model.predict(vectorizer.transform(words_to_check))
        misclassified_samples.extend(
            [
                word
                for word, prediction in zip(words_to_check, predictions)
                if not prediction
            ]
        )

        if len(misclassified_samples) >= misclassification_threshold:
            break

        epochs += 1

    # Save custom dictionary incrementally
    new_vocabulary = vectorizer.get_feature_names_out()
    if os.path.exists(custom_dictionary_filename):
        with open(custom_dictionary_filename, "r") as file:
            existing_vocabulary = set(json.load(file))
        new_vocabulary = existing_vocabulary.union(new_vocabulary)
    with open(custom_dictionary_filename, "w") as file:
        json.dump(list(new_vocabulary), file)

    # Save misclassified samples
    with open(misclassified_samples_filename, "w") as file:
        json.dump(misclassified_samples, file)

    # Check spelling and print corrected words and suggestions
    corrected_words, suggestions = check_spelling(spell, words_to_check)
    for word, suggestion_list in zip(corrected_words, suggestions):
        print(f"Original: {word}, Corrected: {word}, Suggestions: {suggestion_list}")

    print("Training loop finished.")


if __name__ == "__main__":
    main()

Original: today, Corrected: today, Suggestions: {'moldy', 'modal', 'moray', 'mothy', 'moody', 'midday', 'today', 'mayday', 'motley'}
Original: ghost, Corrected: ghost, Suggestions: {'host', 'ghost'}
Original: money, Corrected: money, Suggestions: {'money'}
Training loop finished.
